# **Data Initialization**

**Importing libraries**

In [68]:
import io
import os

import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
import missingno as msno
from sklearn.svm import SVR
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
from matplotlib.backends.backend_pdf import PdfPages
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, accuracy_score
from sklearn.feature_selection import f_regression, RFE, SelectKBest, mutual_info_regression
from keras.models import Sequential 
from keras.layers import Dense 
from keras.optimizers import RMSprop 
from keras.callbacks import EarlyStopping 
from sklearn import preprocessing 
from sklearn.preprocessing import scale

from google.colab import files

**Adding CSV input files after preprocessing**

In [69]:
uploaded = files.upload()

Saving EmoSoundsAfter.csv to EmoSoundsAfter (1).csv
Saving IADSEDAfter.csv to IADSEDAfter (1).csv


**Reading uploaded CSVs and initializing dataframes**

In [108]:
df_emosounds = pd.read_csv(io.BytesIO(uploaded['EmoSoundsAfter.csv']))
df_emosounds_init = df_emosounds.iloc[:, :6]
df_emosounds = df_emosounds.iloc[:, 6:]

df_iadsed = pd.read_csv(io.BytesIO(uploaded['IADSEDAfter.csv']))
df_iadsed_init = df_iadsed.iloc[:, :6]
df_iadsed = df_iadsed.iloc[:, 6:]

# **Train, Test and Validation split**

**Train, test and validation split based on the ratio 80:10:10 for Emosounds dataset**

In [109]:
df_emosounds_train, df_emosounds_test_validation = train_test_split(df_emosounds, test_size=0.2)
df_emosounds_test, df_emosounds_validation = train_test_split(df_emosounds_test_validation, test_size=0.5)

**Train, test and validation split based on the ratio 80:10:10 for IADSED dataset**

In [110]:
df_iadsed_train, df_iadsed_test_validation = train_test_split(df_iadsed, test_size=0.2)
df_iadsed_test, df_iadsed_validation = train_test_split(df_iadsed_test_validation, test_size=0.5)

**Printing shapes of the datframes after splitting**

In [111]:
print(df_iadsed_train.shape, df_iadsed_test.shape, df_iadsed_validation.shape)
print(df_emosounds_train.shape, df_emosounds_test.shape, df_emosounds_validation.shape)

(508, 71) (64, 71) (64, 71)
(325, 70) (41, 70) (41, 70)


**Dividing the data into input(features) and target classes for Emosounds dataframe for train, test and validation sets**

In [112]:
df_emosounds_train_input = df_emosounds_train.iloc[:, 2:]
df_emosounds_train_actual_arousal = df_emosounds_train.iloc[:, 0]
df_emosounds_train_actual_valence = df_emosounds_train.iloc[:, 1]

df_emosounds_test_input = df_emosounds_test.iloc[:, 2:]
df_emosounds_test_actual_arousal = df_emosounds_test.iloc[:, 0]
df_emosounds_test_actual_valence = df_emosounds_test.iloc[:, 1]

df_emosounds_validation_input = df_emosounds_validation.iloc[:, 2:]
df_emosounds_validation_actual_arousal = df_emosounds_validation.iloc[:, 0]
df_emosounds_validation_actual_valence = df_emosounds_validation.iloc[:, 1]

**Dividing the data into input(features) and target classes for IADSED dataframe for train, test and validation sets**

In [113]:
df_iadsed_train_input = df_iadsed_train.iloc[:, 3:]
df_iadsed_train_actual_arousal = df_iadsed_train.iloc[:, 0]
df_iadsed_train_actual_valence = df_iadsed_train.iloc[:, 1]
df_iadsed_train_actual_dominance = df_iadsed_train.iloc[:, 2]

df_iadsed_test_input = df_iadsed_test.iloc[:, 3:]
df_iadsed_test_actual_arousal = df_iadsed_test.iloc[:, 0]
df_iadsed_test_actual_valence = df_iadsed_test.iloc[:, 1]
df_iadsed_test_actual_dominance = df_iadsed_test.iloc[:, 2]

df_iadsed_validation_input = df_iadsed_validation.iloc[:, 3:]
df_iadsed_validation_actual_arousal = df_iadsed_validation.iloc[:, 0]
df_iadsed_validation_actual_valence = df_iadsed_validation.iloc[:, 1]
df_iadsed_validation_actual_dominance = df_iadsed_validation.iloc[:, 2]

# **Training Regression models**

### **Linear Regression**

**Method to train the Linear Regression model with Train set and validate using validation set**

In [76]:
def apply_lr(X_train, y_train, X_test, y_test):
  LR = LinearRegression()
  LR.fit(X_train, y_train)
  predicted = LR.predict(X_test)

  rmse = mean_squared_error(y_test, predicted) ** 0.5
  mae = mean_absolute_error(y_test, predicted)
  
  print("Root Mean Squared Error", rmse)
  print("Mean Absolute Error", mae)

  return rmse, mae, LR

**Training model for datasets - Emosounds and IADSED**

In [ ]:
# LR For emosounds arousal
print("\nLinear Regression\n")
print("For emosounds arousal")
apply_lr(df_emosounds_train_input, df_emosounds_train_actual_arousal, df_emosounds_validation_input, df_emosounds_validation_actual_arousal)

# LR For emosounds valence
print("\nFor emosounds valence")
apply_lr(df_emosounds_train_input, df_emosounds_train_actual_valence, df_emosounds_validation_input, df_emosounds_validation_actual_valence)

# LR For iadsed arousal
print("\nFor iadsed arousal")
apply_lr(df_iadsed_train_input, df_iadsed_train_actual_arousal, df_iadsed_validation_input, df_iadsed_validation_actual_arousal)

# LR For iadsed valence
print("\nFor iadsed valence")
apply_lr(df_iadsed_train_input, df_iadsed_train_actual_valence, df_iadsed_validation_input, df_iadsed_validation_actual_valence)

# LR For iadsed dominance
print("\nFor iadsed dominance")
apply_lr(df_iadsed_train_input, df_iadsed_train_actual_dominance, df_iadsed_validation_input, df_iadsed_validation_actual_dominance)

**Method to train model using kfold cross validation technique**

In [78]:
#With k-Fold cv

def apply_lr_with_kfold(input_df, y):
  lr_kf = KFold(n_splits=5)
  lr_kf.get_n_splits(input_df)
  print(lr_kf)
  sum_rmse_error = 0
  sum_mae_error = 0

  for i, (train_index, test_index) in enumerate(lr_kf.split(input_df)):
    print(i)
    df_input_train , df_input_test = input_df.iloc[train_index,:], input_df.iloc[test_index,:]
    df_train , df_test = y.iloc[train_index] , y.iloc[test_index]
    curr_rmse, curr_mae, model = apply_lr(df_input_train, df_train, df_input_test, df_test)

    sum_rmse_error += curr_rmse
    sum_mae_error += curr_mae

  print("\nAverage Root Mean Squared Error", sum_rmse_error/5)
  print("Average Mean Absolute Error", sum_mae_error/5)
  
  return model

**Method to test the Linear Regression model with test set**

In [79]:
def test_lr_with_kfold(model, X_test, y_test):
  predicted = model.predict(X_test)

  rmse = mean_squared_error(y_test, predicted) ** 0.5
  mae = mean_absolute_error(y_test, predicted)
  
  print("Final Results for Root Mean Squared Error", rmse)
  print("Final Results for Mean Absolute Error", mae)

**Fitting(training) the linear regression model and then testing the same model using kfold cross validation technique for Emosounds and IADSED dataset**

In [80]:
#K-Fold cv for emosounds arousal
print("\n With K-Fold CV")
print("For emosounds arousal")
fitted_model = apply_lr_with_kfold(df_emosounds_train_input, df_emosounds_train_actual_arousal)
test_lr_with_kfold(fitted_model, df_emosounds_test_input, df_emosounds_test_actual_arousal)

#K-Fold cv for emosounds valence
print("\nFor emosounds valence")
fitted_model = apply_lr_with_kfold(df_emosounds_train_input, df_emosounds_train_actual_valence)
test_lr_with_kfold(fitted_model, df_emosounds_test_input, df_emosounds_test_actual_valence)

#K-Fold cv for iadsed arousal
print("\nFor iadsed arousal")
fitted_model = apply_lr_with_kfold(df_iadsed_train_input, df_iadsed_train_actual_arousal)
test_lr_with_kfold(fitted_model, df_iadsed_test_input, df_iadsed_test_actual_arousal)

#K-Fold cv for iadsed valence
print("\nFor iadsed valence")
fitted_model = apply_lr_with_kfold(df_iadsed_train_input, df_iadsed_train_actual_valence)
test_lr_with_kfold(fitted_model, df_iadsed_test_input, df_iadsed_test_actual_valence)

#K-Fold cv for iadsed dominance
print("\nFor iadsed dominance")
fitted_model = apply_lr_with_kfold(df_iadsed_train_input, df_iadsed_train_actual_dominance)
test_lr_with_kfold(fitted_model, df_iadsed_test_input, df_iadsed_test_actual_dominance)


 With K-Fold CV
For emosounds arousal
KFold(n_splits=5, random_state=None, shuffle=False)
0
Root Mean Squared Error 0.2421907671454805
Mean Absolute Error 0.19504689140850276
1
Root Mean Squared Error 0.23584667348663022
Mean Absolute Error 0.1954874172958009
2
Root Mean Squared Error 0.19804678481216811
Mean Absolute Error 0.15895436948968134
3
Root Mean Squared Error 0.22320935982141796
Mean Absolute Error 0.1820870763288223
4
Root Mean Squared Error 0.25241173769425346
Mean Absolute Error 0.21150120214139273

Average Root Mean Squared Error 0.23034106459199005
Average Mean Absolute Error 0.18861539133283997
Final Results for Root Mean Squared Error 0.21907129014145477
Final Results for Mean Absolute Error 0.17698698710305155

For emosounds valence
KFold(n_splits=5, random_state=None, shuffle=False)
0
Root Mean Squared Error 0.40226922901861756
Mean Absolute Error 0.31435597286145867
1
Root Mean Squared Error 0.42071217673559913
Mean Absolute Error 0.35382053692395504
2
Root Mean Sq

### **Support Vector Regression**

**Method to train the Support Vector Regression model with Train set and validate using validation set**

In [114]:
def apply_svr(X_train, y_train, X_test, y_test):

  svr_model = SVR(kernel='linear')
  svr_model.fit(X_train,y_train)
  
  SVR_predicted = svr_model.predict(X_test)

  rmse = mean_squared_error(y_test, SVR_predicted) ** 0.5
  mae = mean_absolute_error(y_test, SVR_predicted)

  print("Root Mean Squared Error", rmse)
  print("Mean Absolute Error", mae)

  return rmse, mae, svr_model


**Training model for datasets - Emosounds and IADSED**

In [115]:
#For emosounds

#For Arousal
print("For emosounds arousal:")
apply_svr(df_emosounds_train_input, df_emosounds_train_actual_arousal, df_emosounds_validation_input, df_emosounds_validation_actual_arousal)

#For Valence
print("\nFor emosounds valence:")
apply_svr(df_emosounds_train_input, df_emosounds_train_actual_valence, df_emosounds_validation_input, df_emosounds_validation_actual_valence)

#For iadsed

#For Arousal
print("\nFor iadsed arousal:")
apply_svr(df_iadsed_train_input, df_iadsed_train_actual_arousal, df_iadsed_test_input, df_iadsed_test_actual_arousal)

#For Valence
print("\nFor iadsed valence:")
apply_svr(df_iadsed_train_input, df_iadsed_train_actual_valence, df_iadsed_test_input, df_iadsed_test_actual_valence)

#For Valence
print("\nFor iadsed dominance:")
apply_svr(df_iadsed_train_input, df_iadsed_train_actual_dominance, df_iadsed_test_input, df_iadsed_test_actual_dominance)

For emosounds arousal:
Root Mean Squared Error 0.2144746740137168
Mean Absolute Error 0.1757584419009493

For emosounds valence:
Root Mean Squared Error 0.38184613757288294
Mean Absolute Error 0.30818235051631926

For iadsed arousal:
Root Mean Squared Error 0.8709302157420084
Mean Absolute Error 0.6626968816799549

For iadsed valence:
Root Mean Squared Error 1.0526704260310806
Mean Absolute Error 0.8438108582209861

For iadsed dominance:
Root Mean Squared Error 0.815717798632619
Mean Absolute Error 0.6429908645815087


(0.815717798632619, 0.6429908645815087, SVR(kernel='linear'))

**Method to train Support Vector Regression model using kfold cross validation technique**

In [116]:
#With k-Fold cv

def apply_svr_with_kfold(input_df, y):
  svr_kf = KFold(n_splits=5)
  svr_kf.get_n_splits(input_df)
  print(svr_kf)
  sum_rmse_error = 0
  sum_mae_error = 0

  for i, (train_index, test_index) in enumerate(svr_kf.split(input_df)):
    print(i)
    df_input_train , df_input_test = input_df.iloc[train_index,:], input_df.iloc[test_index,:]
    df_train , df_test = y.iloc[train_index] , y.iloc[test_index]
    curr_rmse, curr_mae, model = apply_svr(df_input_train, df_train, df_input_test, df_test)

    sum_rmse_error += curr_rmse
    sum_mae_error += curr_mae

  print("\nAverage Root Mean Squared Error", sum_rmse_error/5)
  print("Average Mean Absolute Error", sum_mae_error/5)
  
  return model

**Method to test the Support Vector Regression model with test set**

In [117]:
def test_svr_with_kfold(model, X_test, y_test):
  predicted = model.predict(X_test)

  rmse = mean_squared_error(y_test, predicted) ** 0.5
  mae = mean_absolute_error(y_test, predicted)
  
  print("Final Results for Root Mean Squared Error", rmse)
  print("Final Results for Mean Absolute Error", mae)

**Fitting(training) the support vector regression model and then testing the same model using kfold cross validation technique for Emosounds and IADSED dataset**

In [118]:
#K-Fold cv for emosounds arousal
print("\nWith K-Fold CV")
print("\nFor emosounds arousal")
fitted_model = apply_svr_with_kfold(df_emosounds_train_input, df_emosounds_train_actual_arousal)
test_svr_with_kfold(fitted_model, df_emosounds_test_input, df_emosounds_test_actual_arousal)

#K-Fold cv for emosounds valence
print("\nFor emosounds valence")
fitted_model = apply_svr_with_kfold(df_emosounds_train_input, df_emosounds_train_actual_valence)
test_svr_with_kfold(fitted_model, df_emosounds_test_input, df_emosounds_test_actual_valence)

#K-Fold cv for iadsed arousal
print("\nFor iadsed arousal")
fitted_model = apply_svr_with_kfold(df_iadsed_train_input, df_iadsed_train_actual_arousal)
test_svr_with_kfold(fitted_model, df_iadsed_test_input, df_iadsed_test_actual_arousal)

#K-Fold cv for iadsed valence
print("\nFor iadsed valence")
fitted_model = apply_svr_with_kfold(df_iadsed_train_input, df_iadsed_train_actual_valence)
test_svr_with_kfold(fitted_model, df_iadsed_test_input, df_iadsed_test_actual_valence)

#K-Fold cv for iadsed dominance
print("\nFor iadsed dominance")
fitted_model = apply_svr_with_kfold(df_iadsed_train_input, df_iadsed_train_actual_dominance)
test_svr_with_kfold(fitted_model, df_iadsed_test_input, df_iadsed_test_actual_dominance)


With K-Fold CV

For emosounds arousal
KFold(n_splits=5, random_state=None, shuffle=False)
0
Root Mean Squared Error 0.2353626130807659
Mean Absolute Error 0.18868578357420676
1
Root Mean Squared Error 0.2501835791021092
Mean Absolute Error 0.19632175517897774
2
Root Mean Squared Error 0.22579797506355687
Mean Absolute Error 0.18401320966848236
3
Root Mean Squared Error 0.2613454323361837
Mean Absolute Error 0.2032416544682872
4
Root Mean Squared Error 0.22434091373310108
Mean Absolute Error 0.18607062979088074

Average Root Mean Squared Error 0.23940610266314338
Average Mean Absolute Error 0.19166660653616696
Final Results for Root Mean Squared Error 0.1861665460914345
Final Results for Mean Absolute Error 0.15291947278327273

For emosounds valence
KFold(n_splits=5, random_state=None, shuffle=False)
0
Root Mean Squared Error 0.40729434135413173
Mean Absolute Error 0.3189428527480996
1
Root Mean Squared Error 0.42355481996877586
Mean Absolute Error 0.3324341081712186
2
Root Mean Square

# **Deep Learning**

**Method to train the Deep Learning model with Train set and validate using validation set**  
We have used the sequential model with 3 layers each having 64 neurons.  
The number of epochs are 500 with an activation function ReLu.

In [28]:
def apply_deep_model(x_train, y_train, x_test, y_test, variables):  
  model = Sequential() 
  model.add(Dense(64, kernel_initializer = 'normal', activation = 'relu',
  input_shape = (variables,))) 
  model.add(Dense(64, activation = 'relu')) 
  model.add(Dense(1))

  model.compile(loss = 'mse', optimizer = RMSprop(), metrics = ['mean_squared_error','mean_absolute_error'])

  history = model.fit(
    x_train, y_train,    
    batch_size=128, 
    epochs = 500, 
    verbose = 1, 
    validation_split = 0.2, 
    callbacks = [EarlyStopping(monitor = 'val_loss', patience = 20)]
  )

  predicted = model.predict(x_test) 

  rmse = mean_squared_error(y_test, predicted) ** 0.5
  mae = mean_absolute_error(y_test, predicted)
    
  print("Final Results for Root Mean Squared Error", rmse)
  print("Final Results for Mean Absolute Error", mae)

  return rmse, mae, model

**Training model for datasets - Emosounds and IADSED**

In [ ]:
#For emosounds

#For Arousal
print("For emosounds arousal:")
apply_deep_model(df_emosounds_train_input, df_emosounds_train_actual_arousal, df_emosounds_validation_input, df_emosounds_validation_actual_arousal, 68)

#For Valence
print("\nFor emosounds valence:")
apply_deep_model(df_emosounds_train_input, df_emosounds_train_actual_valence, df_emosounds_validation_input, df_emosounds_validation_actual_valence, 68)

#For iadsed

#For Arousal
print("\nFor iadsed arousal:")
apply_deep_model(df_iadsed_train_input, df_iadsed_train_actual_arousal, df_iadsed_test_input, df_iadsed_test_actual_arousal, 68)

#For Valence
print("\nFor iadsed valence:")
apply_deep_model(df_iadsed_train_input, df_iadsed_train_actual_valence, df_iadsed_test_input, df_iadsed_test_actual_valence, 68)

#For Valence
print("\nFor iadsed dominance:")
apply_deep_model(df_iadsed_train_input, df_iadsed_train_actual_dominance, df_iadsed_test_input, df_iadsed_test_actual_dominance, 68)

**Method to train the Deep Learning model using kfold cross validation technique**

In [29]:
#With k-Fold cv

def apply_deep_model_with_kfold(input_df, y, variables):
  deep_model_kf = KFold(n_splits=5)
  deep_model_kf.get_n_splits(input_df)
  # print(deep_model_kf)
  sum_rmse_error = 0
  sum_mae_error = 0

  for i, (train_index, test_index) in enumerate(deep_model_kf.split(input_df)):
    print(i)
    df_input_train , df_input_test = input_df.iloc[train_index,:], input_df.iloc[test_index,:]
    df_train , df_test = y.iloc[train_index] , y.iloc[test_index]
    curr_rmse, curr_mae, model = apply_deep_model(df_input_train, df_train, df_input_test, df_test, variables)

    sum_rmse_error += curr_rmse
    sum_mae_error += curr_mae

  print("\nAverage Root Mean Squared Error", sum_rmse_error/5)
  print("Average Mean Absolute Error", sum_mae_error/5)
  
  return model

**Method to test the Deep Learning model with test set**

In [30]:
def test_deep_model_with_kfold(model, X_test, y_test):
  predicted = model.predict(X_test)

  rmse = mean_squared_error(y_test, predicted) ** 0.5
  mae = mean_absolute_error(y_test, predicted)
  print("########################################################################################################################")
  print("Final Results for Root Mean Squared Error", rmse)
  print("Final Results for Mean Absolute Error", mae)
  print("########################################################################################################################")

**Fitting(training) the deep learning model and then testing the same model using kfold cross validation technique for Emosounds and IADSED dataset**

In [31]:
#For emosounds

#For Arousal
print("For emosounds arousal:")
model = apply_deep_model_with_kfold(df_emosounds_train_input, df_emosounds_train_actual_arousal, 68)
test_deep_model_with_kfold(model, df_emosounds_test_input, df_emosounds_test_actual_arousal)

#For Valence
print("\nFor emosounds valence:")
model = apply_deep_model_with_kfold(df_emosounds_train_input, df_emosounds_train_actual_valence, 68)
test_deep_model_with_kfold(model, df_emosounds_test_input, df_emosounds_test_actual_valence)

#For iadsed

#For Arousal
print("\nFor iadsed arousal:")
model = apply_deep_model_with_kfold(df_iadsed_train_input, df_iadsed_train_actual_arousal, 68)
test_deep_model_with_kfold(model, df_iadsed_test_input, df_iadsed_test_actual_arousal)

#For Valence
print("\nFor iadsed valence:")
model = apply_deep_model_with_kfold(df_iadsed_train_input, df_iadsed_train_actual_valence, 68)
test_deep_model_with_kfold(model, df_iadsed_test_input, df_iadsed_test_actual_valence)

#For Valence
print("\nFor iadsed dominance:")
model = apply_deep_model_with_kfold(df_iadsed_train_input, df_iadsed_train_actual_dominance, 68)
test_deep_model_with_kfold(model, df_iadsed_test_input, df_iadsed_test_actual_dominance)

For emosounds arousal:
0
Epoch 1/500
2/2 [==============================] - 1s 254ms/step - loss: 0.3639 - mean_squared_error: 0.3639 - mean_absolute_error: 0.5160 - val_loss: 0.1613 - val_mean_squared_error: 0.1613 - val_mean_absolute_error: 0.3371
Epoch 2/500
2/2 [==============================] - 0s 37ms/step - loss: 0.1599 - mean_squared_error: 0.1599 - mean_absolute_error: 0.3286 - val_loss: 0.1171 - val_mean_squared_error: 0.1171 - val_mean_absolute_error: 0.2864
Epoch 3/500
2/2 [==============================] - 0s 34ms/step - loss: 0.1011 - mean_squared_error: 0.1011 - mean_absolute_error: 0.2662 - val_loss: 0.0866 - val_mean_squared_error: 0.0866 - val_mean_absolute_error: 0.2449
Epoch 4/500
2/2 [==============================] - 0s 35ms/step - loss: 0.0743 - mean_squared_error: 0.0743 - mean_absolute_error: 0.2225 - val_loss: 0.0785 - val_mean_squared_error: 0.0785 - val_mean_absolute_error: 0.2375
Epoch 5/500
2/2 [==============================] - 0s 53ms/step - loss: 0.0585

3/3 [==============================] - 0s 5ms/step
Final Results for Root Mean Squared Error 0.2760758027368731
Final Results for Mean Absolute Error 0.2199195867071552

Average Root Mean Squared Error 0.2846126276836719
Average Mean Absolute Error 0.22295717456942712
2/2 [==============================] - 0s 7ms/step
########################################################################################################################
Final Results for Root Mean Squared Error 0.25609255163902367
Final Results for Mean Absolute Error 0.20837011271309297
########################################################################################################################

For emosounds valence:
0
Epoch 1/500
2/2 [==============================] - 1s 195ms/step - loss: 0.3366 - mean_squared_error: 0.3366 - mean_absolute_error: 0.4931 - val_loss: 0.2227 - val_mean_squared_error: 0.2227 - val_mean_absolute_error: 0.3808
Epoch 2/500
2/2 [==============================] - 0s 39ms/step - l

3/3 [==============================] - 0s 5ms/step
Final Results for Root Mean Squared Error 0.3541076873467069
Final Results for Mean Absolute Error 0.27428470293984464
4
Epoch 1/500
2/2 [==============================] - 1s 195ms/step - loss: 0.4197 - mean_squared_error: 0.4197 - mean_absolute_error: 0.5447 - val_loss: 0.1809 - val_mean_squared_error: 0.1809 - val_mean_absolute_error: 0.3498
Epoch 2/500
2/2 [==============================] - 0s 37ms/step - loss: 0.2167 - mean_squared_error: 0.2167 - mean_absolute_error: 0.3965 - val_loss: 0.1600 - val_mean_squared_error: 0.1600 - val_mean_absolute_error: 0.3302
Epoch 3/500
2/2 [==============================] - 0s 39ms/step - loss: 0.1604 - mean_squared_error: 0.1604 - mean_absolute_error: 0.3383 - val_loss: 0.1450 - val_mean_squared_error: 0.1450 - val_mean_absolute_error: 0.3127
Epoch 4/500
2/2 [==============================] - 0s 62ms/step - loss: 0.1297 - mean_squared_error: 0.1297 - mean_absolute_error: 0.3015 - val_loss: 0.137

# **Feature Selection**

## **F Regression**

**Testing the f regression as a technique for feature selection**

In [ ]:
df_emosounds = pd.read_csv(io.BytesIO(uploaded['EmoSoundsAfter.csv']))
df_emosounds_input = df_emosounds.iloc[:, 8:]
df_emosounds_target_arousal = df_emosounds.iloc[:, 6]
df_emosounds_target_valence = df_emosounds.iloc[:, 7]

columns = df_emosounds_input.columns
select = SelectKBest(score_func=f_regression, k=48)
df_emosounds_input_arousal = select.fit_transform(df_emosounds_input, df_emosounds_target_arousal)
columns = columns[select.get_support()]
df_emosounds_input_arousal = pd.DataFrame(df_emosounds_input_arousal, columns = columns)

df_emosounds_arousal = pd.concat([df_emosounds_target_arousal, df_emosounds_target_valence], axis=1)
df_emosounds = pd.concat([df_emosounds_arousal, df_emosounds_input_arousal], axis=1)
df_emosounds_train, df_emosounds_test_validation = train_test_split(df_emosounds, test_size=0.2)
df_emosounds_test, df_emosounds_validation = train_test_split(df_emosounds_test_validation, test_size=0.5)
print(df_emosounds_train.shape)
print(df_emosounds_test.shape)
print(df_emosounds_validation.shape)

df_emosounds_train_input = df_emosounds_train.iloc[:, 2:]
df_emosounds_train_actual_arousal = df_emosounds_train.iloc[:, 0]

df_emosounds_validation_input = df_emosounds_validation.iloc[:, 2:]
df_emosounds_validation_actual_arousal = df_emosounds_validation.iloc[:, 0]

# LR For emosounds arousal
print("\nLinear Regression\n")
print("For emosounds arousal")
apply_lr(df_emosounds_train_input, df_emosounds_train_actual_arousal, df_emosounds_validation_input, df_emosounds_validation_actual_arousal)

(325, 50)
(41, 50)
(41, 50)

Linear Regression

For emosounds arousal
Root Mean Squared Error 0.2056034822373003
Mean Absolute Error 0.16229554968165158


(0.2056034822373003, 0.16229554968165158, LinearRegression())

## **Mutual Information Regression**

**Testing the mutual information regression as a technique for feature selection**

In [ ]:
df_emosounds = pd.read_csv(io.BytesIO(uploaded['EmoSoundsAfter.csv']))
df_emosounds_input = df_emosounds.iloc[:, 8:]
df_emosounds_target_arousal = df_emosounds.iloc[:, 6]
df_emosounds_target_valence = df_emosounds.iloc[:, 7]

columns = df_emosounds_input.columns
select = SelectKBest(score_func=mutual_info_regression, k=48)
df_emosounds_input_arousal = select.fit_transform(df_emosounds_input, df_emosounds_target_arousal)
columns = columns[select.get_support()]
df_emosounds_input_arousal = pd.DataFrame(df_emosounds_input_arousal, columns = columns)

df_emosounds_arousal = pd.concat([df_emosounds_target_arousal, df_emosounds_target_valence], axis=1)
df_emosounds = pd.concat([df_emosounds_arousal, df_emosounds_input_arousal], axis=1)
df_emosounds_train, df_emosounds_test_validation = train_test_split(df_emosounds, test_size=0.2)
df_emosounds_test, df_emosounds_validation = train_test_split(df_emosounds_test_validation, test_size=0.5)
print(df_emosounds_train.shape)
print(df_emosounds_test.shape)
print(df_emosounds_validation.shape)

df_emosounds_train_input = df_emosounds_train.iloc[:, 2:]
df_emosounds_train_actual_arousal = df_emosounds_train.iloc[:, 0]

df_emosounds_validation_input = df_emosounds_validation.iloc[:, 2:]
df_emosounds_validation_actual_arousal = df_emosounds_validation.iloc[:, 0]

# LR For emosounds arousal
print("\nLinear Regression\n")
print("For emosounds arousal")
apply_lr(df_emosounds_train_input, df_emosounds_train_actual_arousal, df_emosounds_validation_input, df_emosounds_validation_actual_arousal)

(325, 50)
(41, 50)
(41, 50)

Linear Regression

For emosounds arousal
Root Mean Squared Error 0.18637693963397495
Mean Absolute Error 0.14478116257088086


(0.18637693963397495, 0.14478116257088086, LinearRegression())

## **Recursive Feature Elimination**

**Testing the recursive feature elimination as a technique for feature selection**

In [ ]:
df_emosounds = pd.read_csv(io.BytesIO(uploaded['EmoSoundsAfter.csv']))
df_emosounds_input = df_emosounds.iloc[:, 8:]
df_emosounds_target_arousal = df_emosounds.iloc[:, 6]
df_emosounds_target_valence = df_emosounds.iloc[:, 7]

columns = df_emosounds_input.columns
estimator = LinearRegression()
rfe = RFE(estimator, n_features_to_select=48)
df_emosounds_input_arousal = rfe.fit_transform(df_emosounds_input, df_emosounds_target_arousal)
columns = columns[rfe.support_]
df_emosounds_input_arousal = pd.DataFrame(df_emosounds_input_arousal, columns = columns)

df_emosounds_arousal = pd.concat([df_emosounds_target_arousal, df_emosounds_target_valence], axis=1)
df_emosounds = pd.concat([df_emosounds_arousal, df_emosounds_input_arousal], axis=1)
df_emosounds_train, df_emosounds_test_validation = train_test_split(df_emosounds, test_size=0.2)
df_emosounds_test, df_emosounds_validation = train_test_split(df_emosounds_test_validation, test_size=0.5)
print(df_emosounds_train.shape)
print(df_emosounds_test.shape)
print(df_emosounds_validation.shape)

df_emosounds_train_input = df_emosounds_train.iloc[:, 2:]
df_emosounds_train_actual_arousal = df_emosounds_train.iloc[:, 0]

df_emosounds_validation_input = df_emosounds_validation.iloc[:, 2:]
df_emosounds_validation_actual_arousal = df_emosounds_validation.iloc[:, 0]

# LR For emosounds arousal
print("\nLinear Regression\n")
print("For emosounds arousal")
apply_lr(df_emosounds_train_input, df_emosounds_train_actual_arousal, df_emosounds_validation_input, df_emosounds_validation_actual_arousal)

(325, 50)
(41, 50)
(41, 50)

Linear Regression

For emosounds arousal
Root Mean Squared Error 0.17913247013107494
Mean Absolute Error 0.14538474342509078


(0.17913247013107494, 0.14538474342509078, LinearRegression())

## **Final models after Feature Selection for Emosounds**

### **Final model after Feature Selection with respect to Arousal**

**Splitting data for emosounds datset**

In [81]:
df_emosounds = pd.read_csv(io.BytesIO(uploaded['EmoSoundsAfter.csv']))
df_emosounds_input = df_emosounds.iloc[:, 8:]
df_emosounds_target_arousal = df_emosounds.iloc[:, 6]
df_emosounds_target_valence = df_emosounds.iloc[:, 7]

**Selecting top 48 features based on the RFE feature selection technique with respect to target class arousal**

In [84]:
# select = SelectKBest(score_func=f_regression, k=48)
# select = SelectKBest(score_func=mutual_info_regression, k=48)
# df_emosounds_input_arousal = select.fit_transform(df_emosounds_input, df_emosounds_target_arousal)

columns = df_emosounds_input.columns
estimator = LinearRegression()
rfe = RFE(estimator, n_features_to_select=48)
df_emosounds_input_arousal = rfe.fit_transform(df_emosounds_input, df_emosounds_target_arousal)
columns = columns[rfe.support_]
# columns = columns[select.get_support()]
df_emosounds_input_arousal = pd.DataFrame(df_emosounds_input_arousal, columns = columns)

**Splitting data into train, test, validation sets**

In [85]:
df_emosounds_arousal = pd.concat([df_emosounds_target_arousal, df_emosounds_target_valence], axis=1)
df_emosounds = pd.concat([df_emosounds_arousal, df_emosounds_input_arousal], axis=1)
df_emosounds_train, df_emosounds_test_validation = train_test_split(df_emosounds, test_size=0.2)
df_emosounds_test, df_emosounds_validation = train_test_split(df_emosounds_test_validation, test_size=0.5)
print(df_emosounds_train.shape)
print(df_emosounds_test.shape)
print(df_emosounds_validation.shape)

(325, 50)
(41, 50)
(41, 50)


In [86]:
df_emosounds_train_input = df_emosounds_train.iloc[:, 2:]
df_emosounds_train_actual_arousal = df_emosounds_train.iloc[:, 0]

df_emosounds_test_input = df_emosounds_test.iloc[:, 2:]
df_emosounds_test_actual_arousal = df_emosounds_test.iloc[:, 0]

df_emosounds_validation_input = df_emosounds_validation.iloc[:, 2:]
df_emosounds_validation_actual_arousal = df_emosounds_validation.iloc[:, 0]

**Fitting(training) the models and then testing the same model using kfold cross validation technique for Emosounds arousal dataset after feature selection**

In [87]:
#K-Fold cv for emosounds arousal
print("\n With K-Fold CV")
print("For emosounds arousal LR")
fitted_model = apply_lr_with_kfold(df_emosounds_train_input, df_emosounds_train_actual_arousal)
test_lr_with_kfold(fitted_model, df_emosounds_test_input, df_emosounds_test_actual_arousal)

print("\n With K-Fold CV")
print("For emosounds arousal SVR")
fitted_model = apply_svr_with_kfold(df_emosounds_train_input, df_emosounds_train_actual_arousal)
test_svr_with_kfold(fitted_model, df_emosounds_test_input, df_emosounds_test_actual_arousal)


 With K-Fold CV
For emosounds arousal LR
KFold(n_splits=5, random_state=None, shuffle=False)
0
Root Mean Squared Error 0.20851077106367286
Mean Absolute Error 0.16699962769604715
1
Root Mean Squared Error 0.21356628807177852
Mean Absolute Error 0.17494243843821877
2
Root Mean Squared Error 0.23372536910614805
Mean Absolute Error 0.17228049532114573
3
Root Mean Squared Error 0.22969744868631034
Mean Absolute Error 0.18835933910624778
4
Root Mean Squared Error 0.19580906161106915
Mean Absolute Error 0.16206442910021687

Average Root Mean Squared Error 0.21626178770779578
Average Mean Absolute Error 0.1729292659323753
Final Results for Root Mean Squared Error 0.1701431906631873
Final Results for Mean Absolute Error 0.13829252516133847

 With K-Fold CV
For emosounds arousal SVR
KFold(n_splits=5, random_state=None, shuffle=False)
0
Root Mean Squared Error 0.21165969644543842
Mean Absolute Error 0.17304475155127544
1
Root Mean Squared Error 0.22602674201656828
Mean Absolute Error 0.18912512

In [37]:
print("\n With K-Fold CV")
print("For emosounds arousal Deep Model")
fitted_model = apply_deep_model_with_kfold(df_emosounds_train_input, df_emosounds_train_actual_arousal, 48)
test_deep_model_with_kfold(fitted_model, df_emosounds_test_input, df_emosounds_test_actual_arousal)


 With K-Fold CV
For emosounds arousal Deep Model
0
Epoch 1/500
2/2 [==============================] - 1s 222ms/step - loss: 0.3988 - mean_squared_error: 0.3988 - mean_absolute_error: 0.5036 - val_loss: 0.2580 - val_mean_squared_error: 0.2580 - val_mean_absolute_error: 0.4324
Epoch 2/500
2/2 [==============================] - 0s 38ms/step - loss: 0.1836 - mean_squared_error: 0.1836 - mean_absolute_error: 0.3586 - val_loss: 0.1989 - val_mean_squared_error: 0.1989 - val_mean_absolute_error: 0.3769
Epoch 3/500
2/2 [==============================] - 0s 36ms/step - loss: 0.1112 - mean_squared_error: 0.1112 - mean_absolute_error: 0.2785 - val_loss: 0.1480 - val_mean_squared_error: 0.1480 - val_mean_absolute_error: 0.3202
Epoch 4/500
2/2 [==============================] - 0s 35ms/step - loss: 0.0777 - mean_squared_error: 0.0777 - mean_absolute_error: 0.2291 - val_loss: 0.1289 - val_mean_squared_error: 0.1289 - val_mean_absolute_error: 0.2931
Epoch 5/500
2/2 [==============================] - 

### **Final model after Feature Selection with respect to Valence**

**Splitting data for emosounds datset**

In [88]:
df_emosounds = pd.read_csv(io.BytesIO(uploaded['EmoSoundsAfter.csv']))
df_emosounds_input = df_emosounds.iloc[:, 8:]
df_emosounds_target_arousal = df_emosounds.iloc[:, 6]
df_emosounds_target_valence = df_emosounds.iloc[:, 7]

**Selecting top 48 features based on the RFE feature selection technique with respect to target class valence**

In [89]:
# select = SelectKBest(score_func=f_regression, k=48)
# select = SelectKBest(score_func=mutual_info_regression, k=48)
# df_emosounds_input_valence = select.fit_transform(df_emosounds_input, df_emosounds_target_valence)

columns = df_emosounds_input.columns
estimator = LinearRegression()
rfe = RFE(estimator, n_features_to_select=48)
df_emosounds_input_valence = rfe.fit_transform(df_emosounds_input, df_emosounds_target_valence)
columns = columns[rfe.support_]
df_emosounds_input_valence = pd.DataFrame(df_emosounds_input_valence, columns = columns)

**Splitting data into train, test, validation sets**

In [90]:
df_emosounds_valence = pd.concat([df_emosounds_target_arousal, df_emosounds_target_valence], axis=1)
df_emosounds = pd.concat([df_emosounds_valence, df_emosounds_input_valence], axis=1)
df_emosounds_train, df_emosounds_test_validation = train_test_split(df_emosounds, test_size=0.2)
df_emosounds_test, df_emosounds_validation = train_test_split(df_emosounds_test_validation, test_size=0.5)
print(df_emosounds_train.shape)
print(df_emosounds_test.shape)
print(df_emosounds_validation.shape)

(325, 50)
(41, 50)
(41, 50)


In [91]:
df_emosounds_train_input = df_emosounds_train.iloc[:, 2:]
df_emosounds_train_actual_valence = df_emosounds_train.iloc[:, 1]

df_emosounds_test_input = df_emosounds_test.iloc[:, 2:]
df_emosounds_test_actual_valence = df_emosounds_test.iloc[:, 1]

df_emosounds_validation_input = df_emosounds_validation.iloc[:, 2:]
df_emosounds_validation_actual_valence = df_emosounds_validation.iloc[:, 1]

**Fitting(training) the models and then testing the same model using kfold cross validation technique for Emosounds valence dataset after feature selection**

In [92]:
#K-Fold cv for emosounds arousal
print("\n With K-Fold CV")
print("For emosounds valence LR")
fitted_model = apply_lr_with_kfold(df_emosounds_train_input, df_emosounds_train_actual_valence)
test_lr_with_kfold(fitted_model, df_emosounds_test_input, df_emosounds_test_actual_valence)

print("\n With K-Fold CV")
print("For emosounds valence SVR")
fitted_model = apply_svr_with_kfold(df_emosounds_train_input, df_emosounds_train_actual_valence)
test_svr_with_kfold(fitted_model, df_emosounds_test_input, df_emosounds_test_actual_valence)


 With K-Fold CV
For emosounds valence LR
KFold(n_splits=5, random_state=None, shuffle=False)
0
Root Mean Squared Error 0.36172172321793733
Mean Absolute Error 0.2799786006224025
1
Root Mean Squared Error 0.3714092091627855
Mean Absolute Error 0.29639270343346297
2
Root Mean Squared Error 0.2816539132728743
Mean Absolute Error 0.23380347068316662
3
Root Mean Squared Error 0.33100637273230793
Mean Absolute Error 0.2723451210383061
4
Root Mean Squared Error 0.4154977583202157
Mean Absolute Error 0.32357547802218645

Average Root Mean Squared Error 0.35225779534122414
Average Mean Absolute Error 0.28121907475990493
Final Results for Root Mean Squared Error 0.43928355771418726
Final Results for Mean Absolute Error 0.35953019431865074

 With K-Fold CV
For emosounds valence SVR
KFold(n_splits=5, random_state=None, shuffle=False)
0
Root Mean Squared Error 0.3564170970444293
Mean Absolute Error 0.2764146671700802
1
Root Mean Squared Error 0.3712615929276206
Mean Absolute Error 0.29289768631869

In [49]:
print("\n With K-Fold CV")
print("For emosounds valence Deep Model")
fitted_model = apply_deep_model_with_kfold(df_emosounds_train_input, df_emosounds_train_actual_valence, 48)
test_deep_model_with_kfold(fitted_model, df_emosounds_test_input, df_emosounds_test_actual_valence)


 With K-Fold CV
For emosounds valence Deep Model
0
Epoch 1/500
2/2 [==============================] - 1s 317ms/step - loss: 0.3041 - mean_squared_error: 0.3041 - mean_absolute_error: 0.4697 - val_loss: 0.2318 - val_mean_squared_error: 0.2318 - val_mean_absolute_error: 0.3923
Epoch 2/500
2/2 [==============================] - 0s 64ms/step - loss: 0.1824 - mean_squared_error: 0.1824 - mean_absolute_error: 0.3597 - val_loss: 0.1929 - val_mean_squared_error: 0.1929 - val_mean_absolute_error: 0.3687
Epoch 3/500
2/2 [==============================] - 0s 45ms/step - loss: 0.1446 - mean_squared_error: 0.1446 - mean_absolute_error: 0.3239 - val_loss: 0.1774 - val_mean_squared_error: 0.1774 - val_mean_absolute_error: 0.3521
Epoch 4/500
2/2 [==============================] - 0s 63ms/step - loss: 0.1176 - mean_squared_error: 0.1176 - mean_absolute_error: 0.2881 - val_loss: 0.1564 - val_mean_squared_error: 0.1564 - val_mean_absolute_error: 0.3305
Epoch 5/500
2/2 [==============================] - 

## **Final models after Feature Selection for IADSED**

### **Final model after Feature Selection with respect to Arousal**

**Splitting data for iadsed datset**

In [93]:
df_iadsed = pd.read_csv(io.BytesIO(uploaded['IADSEDAfter.csv']))
df_iadsed_input = df_iadsed.iloc[:, 9:]
df_iadsed_target_arousal = df_iadsed.iloc[:, 6]
df_iadsed_target_valence = df_iadsed.iloc[:, 7]
df_iadsed_target_dominance = df_iadsed.iloc[:, 8]

**Selecting top 48 features based on the RFE feature selection technique with respect to target class arousal**

In [94]:
# select = SelectKBest(score_func=f_regression, k=48)
# select = SelectKBest(score_func=mutual_info_regression, k=48)
# df_iadsed_input_arousal = select.fit_transform(df_iadsed_input, df_iadsed_target_arousal)

columns = df_iadsed_input.columns
estimator = LinearRegression()
rfe = RFE(estimator, n_features_to_select=48)
df_iadsed_input_arousal = rfe.fit_transform(df_iadsed_input, df_iadsed_target_arousal)
columns = columns[rfe.support_]
# columns = columns[select.get_support()]
df_iadsed_input_arousal = pd.DataFrame(df_iadsed_input_arousal, columns = columns)

**Splitting data into train, test, validation sets**

In [95]:
df_iadsed_arousal = pd.concat([df_iadsed_target_arousal, df_iadsed_target_valence], axis=1)
df_iadsed_arousal = pd.concat([df_iadsed_arousal, df_iadsed_target_dominance], axis=1)
df_iadsed = pd.concat([df_iadsed_arousal, df_iadsed_input_arousal], axis=1)
df_iadsed_train, df_iadsed_test_validation = train_test_split(df_iadsed, test_size=0.2)
df_iadsed_test, df_iadsed_validation = train_test_split(df_iadsed_test_validation, test_size=0.5)
print(df_iadsed_train.shape)
print(df_iadsed_test.shape)
print(df_iadsed_validation.shape)

(508, 51)
(64, 51)
(64, 51)


In [96]:
df_iadsed_train_input = df_iadsed_train.iloc[:, 3:]
df_iadsed_train_actual_arousal = df_iadsed_train.iloc[:, 0]

df_iadsed_test_input = df_iadsed_test.iloc[:, 3:]
df_iadsed_test_actual_arousal = df_iadsed_test.iloc[:, 0]

df_iadsed_validation_input = df_iadsed_validation.iloc[:, 3:]
df_iadsed_validation_actual_arousal = df_iadsed_validation.iloc[:, 0]

**Fitting(training) the models and then testing the same model using kfold cross validation technique for IADSED arousal dataset after feature selection**

In [97]:
#K-Fold cv for iadsed arousal
print("\n With K-Fold CV")
print("For iadsed arousal LR")
fitted_model = apply_lr_with_kfold(df_iadsed_train_input, df_iadsed_train_actual_arousal)
test_lr_with_kfold(fitted_model, df_iadsed_test_input, df_iadsed_test_actual_arousal)

print("\n With K-Fold CV")
print("For iadsed arousal SVR")
fitted_model = apply_svr_with_kfold(df_iadsed_train_input, df_iadsed_train_actual_arousal)
test_svr_with_kfold(fitted_model, df_iadsed_test_input, df_iadsed_test_actual_arousal)


 With K-Fold CV
For iadsed arousal LR
KFold(n_splits=5, random_state=None, shuffle=False)
0
Root Mean Squared Error 0.7820401303083278
Mean Absolute Error 0.5950539670597038
1
Root Mean Squared Error 0.8210392518608319
Mean Absolute Error 0.6581158665192427
2
Root Mean Squared Error 0.8437970689930238
Mean Absolute Error 0.6738674837919972
3
Root Mean Squared Error 0.7962478341548848
Mean Absolute Error 0.639804687432905
4
Root Mean Squared Error 0.7644120853127655
Mean Absolute Error 0.6067379276968127

Average Root Mean Squared Error 0.8015072741259667
Average Mean Absolute Error 0.6347159865001323
Final Results for Root Mean Squared Error 0.6694823037540826
Final Results for Mean Absolute Error 0.5897886182584985

 With K-Fold CV
For iadsed arousal SVR
KFold(n_splits=5, random_state=None, shuffle=False)
0
Root Mean Squared Error 0.7980938470190435
Mean Absolute Error 0.6068978225279101
1
Root Mean Squared Error 0.8810632898495739
Mean Absolute Error 0.6926024630729628
2
Root Mean S

In [55]:
print("\n With K-Fold CV")
print("For iadsed arousal Deep Model")
fitted_model = apply_deep_model_with_kfold(df_iadsed_train_input, df_iadsed_train_actual_arousal, 48)
test_deep_model_with_kfold(fitted_model, df_iadsed_test_input, df_iadsed_test_actual_arousal)


 With K-Fold CV
For iadsed arousal Deep Model
0
Epoch 1/500
3/3 [==============================] - 1s 105ms/step - loss: 31.5475 - mean_squared_error: 31.5475 - mean_absolute_error: 5.4974 - val_loss: 27.8859 - val_mean_squared_error: 27.8859 - val_mean_absolute_error: 5.1556
Epoch 2/500
3/3 [==============================] - 0s 19ms/step - loss: 25.8202 - mean_squared_error: 25.8202 - mean_absolute_error: 4.9491 - val_loss: 23.5411 - val_mean_squared_error: 23.5411 - val_mean_absolute_error: 4.7160
Epoch 3/500
3/3 [==============================] - 0s 18ms/step - loss: 21.5871 - mean_squared_error: 21.5871 - mean_absolute_error: 4.4977 - val_loss: 19.5890 - val_mean_squared_error: 19.5890 - val_mean_absolute_error: 4.2756
Epoch 4/500
3/3 [==============================] - 0s 20ms/step - loss: 17.8167 - mean_squared_error: 17.8167 - mean_absolute_error: 4.0529 - val_loss: 16.0231 - val_mean_squared_error: 16.0231 - val_mean_absolute_error: 3.8287
Epoch 5/500
3/3 [=====================

### **Final model after Feature Selection with respect to Valence**

**Splitting data for iadsed datset**

In [98]:
df_iadsed = pd.read_csv(io.BytesIO(uploaded['IADSEDAfter.csv']))
df_iadsed_input = df_iadsed.iloc[:, 9:]
df_iadsed_target_arousal = df_iadsed.iloc[:, 6]
df_iadsed_target_valence = df_iadsed.iloc[:, 7]
df_iadsed_target_dominance = df_iadsed.iloc[:, 8]

**Selecting top 48 features based on the RFE feature selection technique with respect to target class valence**

In [99]:
# select = SelectKBest(score_func=f_regression, k=48)
# select = SelectKBest(score_func=mutual_info_regression, k=48)
# df_iadsed_input_valence = select.fit_transform(df_iadsed_input, df_iadsed_target_valence)

columns = df_iadsed_input.columns
estimator = LinearRegression()
rfe = RFE(estimator, n_features_to_select=48)
df_iadsed_input_valence = rfe.fit_transform(df_iadsed_input, df_iadsed_target_valence)
columns = columns[rfe.support_]
df_iadsed_input_valence = pd.DataFrame(df_iadsed_input_valence, columns = columns)

**Splitting data into train, test, validation sets**

In [100]:
df_iadsed_arousal = pd.concat([df_iadsed_target_arousal, df_iadsed_target_valence], axis=1)
df_iadsed_arousal = pd.concat([df_iadsed_arousal, df_iadsed_target_dominance], axis=1)
df_iadsed = pd.concat([df_iadsed_arousal, df_iadsed_input_valence], axis=1)
df_iadsed_train, df_iadsed_test_validation = train_test_split(df_iadsed, test_size=0.2)
df_iadsed_test, df_iadsed_validation = train_test_split(df_iadsed_test_validation, test_size=0.5)
print(df_iadsed_train.shape)
print(df_iadsed_test.shape)
print(df_iadsed_validation.shape)

(508, 51)
(64, 51)
(64, 51)


In [101]:
df_iadsed_train_input = df_iadsed_train.iloc[:, 3:]
df_iadsed_train_actual_valence = df_iadsed_train.iloc[:, 1]

df_iadsed_test_input = df_iadsed_test.iloc[:, 3:]
df_iadsed_test_actual_valence = df_iadsed_test.iloc[:, 1]

df_iadsed_validation_input = df_iadsed_validation.iloc[:, 3:]
df_iadsed_validation_actual_valence = df_iadsed_validation.iloc[:, 1]

**Fitting(training) the models and then testing the same model using kfold cross validation technique for IADSED valence dataset after feature selection**

In [102]:
#K-Fold cv for iadsed valence
print("\n With K-Fold CV")
print("For iadsed valence LR")
fitted_model = apply_lr_with_kfold(df_iadsed_train_input, df_iadsed_train_actual_valence)
test_lr_with_kfold(fitted_model, df_iadsed_test_input, df_iadsed_test_actual_valence)

print("\n With K-Fold CV")
print("For iadsed valence SVR")
fitted_model = apply_svr_with_kfold(df_iadsed_train_input, df_iadsed_train_actual_valence)
test_svr_with_kfold(fitted_model, df_iadsed_test_input, df_iadsed_test_actual_valence)


 With K-Fold CV
For iadsed valence LR
KFold(n_splits=5, random_state=None, shuffle=False)
0
Root Mean Squared Error 1.0379270006791013
Mean Absolute Error 0.8482997104154901
1
Root Mean Squared Error 1.272682761611083
Mean Absolute Error 1.033181545762728
2
Root Mean Squared Error 1.3025087387535914
Mean Absolute Error 1.088962138624435
3
Root Mean Squared Error 1.0430983939198422
Mean Absolute Error 0.8344958137170974
4
Root Mean Squared Error 1.2461787069497863
Mean Absolute Error 1.0154063378025617

Average Root Mean Squared Error 1.1804791203826808
Average Mean Absolute Error 0.9640691092644624
Final Results for Root Mean Squared Error 1.2291569085469474
Final Results for Mean Absolute Error 0.9667582760651646

 With K-Fold CV
For iadsed valence SVR
KFold(n_splits=5, random_state=None, shuffle=False)
0
Root Mean Squared Error 1.1562219992071532
Mean Absolute Error 0.9146148917402706
1
Root Mean Squared Error 1.3216133502904288
Mean Absolute Error 1.0856855238416665
2
Root Mean Squ

In [61]:
print("\n With K-Fold CV")
print("For iadsed valence Deep Model")
fitted_model = apply_deep_model_with_kfold(df_iadsed_train_input, df_iadsed_train_actual_valence, 48)
test_deep_model_with_kfold(fitted_model, df_iadsed_test_input, df_iadsed_test_actual_valence)


 With K-Fold CV
For iadsed valence Deep Model
0
Epoch 1/500
3/3 [==============================] - 1s 93ms/step - loss: 23.0909 - mean_squared_error: 23.0909 - mean_absolute_error: 4.5870 - val_loss: 20.2031 - val_mean_squared_error: 20.2031 - val_mean_absolute_error: 4.2649
Epoch 2/500
3/3 [==============================] - 0s 22ms/step - loss: 18.8355 - mean_squared_error: 18.8355 - mean_absolute_error: 4.1027 - val_loss: 17.1024 - val_mean_squared_error: 17.1024 - val_mean_absolute_error: 3.8799
Epoch 3/500
3/3 [==============================] - 0s 19ms/step - loss: 15.6569 - mean_squared_error: 15.6569 - mean_absolute_error: 3.7056 - val_loss: 14.3809 - val_mean_squared_error: 14.3809 - val_mean_absolute_error: 3.5079
Epoch 4/500
3/3 [==============================] - 0s 27ms/step - loss: 12.8884 - mean_squared_error: 12.8884 - mean_absolute_error: 3.3112 - val_loss: 11.9378 - val_mean_squared_error: 11.9378 - val_mean_absolute_error: 3.1414
Epoch 5/500
3/3 [======================

### **Final model after Feature Selection with respect to Dominance**

**Splitting data for iadsed datset**

In [103]:
df_iadsed = pd.read_csv(io.BytesIO(uploaded['IADSEDAfter.csv']))
df_iadsed_input = df_iadsed.iloc[:, 9:]
df_iadsed_target_arousal = df_iadsed.iloc[:, 6]
df_iadsed_target_valence = df_iadsed.iloc[:, 7]
df_iadsed_target_dominance = df_iadsed.iloc[:, 8]

**Selecting top 48 features based on the RFE feature selection technique with respect to target class dominance**

In [104]:
# select = SelectKBest(score_func=f_regression, k=48)
# select = SelectKBest(score_func=mutual_info_regression, k=48)
# df_iadsed_input_dominance = select.fit_transform(df_iadsed_input, df_iadsed_target_dominance)

columns = df_iadsed_input.columns
estimator = LinearRegression()
rfe = RFE(estimator, n_features_to_select=48)
df_iadsed_input_dominance = rfe.fit_transform(df_iadsed_input, df_iadsed_target_dominance)
columns = columns[rfe.support_]
df_iadsed_input_dominance = pd.DataFrame(df_iadsed_input_dominance, columns = columns)

**Splitting data into train, test, validation sets**

In [105]:
df_iadsed_arousal = pd.concat([df_iadsed_target_arousal, df_iadsed_target_valence], axis=1)
df_iadsed_arousal = pd.concat([df_iadsed_arousal, df_iadsed_target_dominance], axis=1)
df_iadsed = pd.concat([df_iadsed_arousal, df_iadsed_input_dominance], axis=1)
df_iadsed_train, df_iadsed_test_validation = train_test_split(df_iadsed, test_size=0.2)
df_iadsed_test, df_iadsed_validation = train_test_split(df_iadsed_test_validation, test_size=0.5)
print(df_iadsed_train.shape)
print(df_iadsed_test.shape)
print(df_iadsed_validation.shape)

(508, 51)
(64, 51)
(64, 51)


In [106]:
df_iadsed_train_input = df_iadsed_train.iloc[:, 3:]
df_iadsed_train_actual_dominance = df_iadsed_train.iloc[:, 2]

df_iadsed_test_input = df_iadsed_test.iloc[:, 3:]
df_iadsed_test_actual_dominance = df_iadsed_test.iloc[:, 2]

df_iadsed_validation_input = df_iadsed_validation.iloc[:, 3:]
df_iadsed_validation_actual_dominance = df_iadsed_validation.iloc[:, 2]

**Fitting(training) the models and then testing the same model using kfold cross validation technique for IADSED dominance dataset after feature selection**

In [107]:
#K-Fold cv for iadsed valence
print("\n With K-Fold CV")
print("For iadsed dominance LR")
fitted_model = apply_lr_with_kfold(df_iadsed_train_input, df_iadsed_train_actual_dominance)
test_lr_with_kfold(fitted_model, df_iadsed_test_input, df_iadsed_test_actual_dominance)

print("\n With K-Fold CV")
print("For iadsed dominance SVR")
fitted_model = apply_svr_with_kfold(df_iadsed_train_input, df_iadsed_train_actual_dominance)
test_svr_with_kfold(fitted_model, df_iadsed_test_input, df_iadsed_test_actual_dominance)


 With K-Fold CV
For iadsed dominance LR
KFold(n_splits=5, random_state=None, shuffle=False)
0
Root Mean Squared Error 0.8987384768301325
Mean Absolute Error 0.7113325708616922
1
Root Mean Squared Error 0.8683213822844126
Mean Absolute Error 0.6837295711134126
2
Root Mean Squared Error 0.7585485430343722
Mean Absolute Error 0.6302133977125857
3
Root Mean Squared Error 0.8886533598982241
Mean Absolute Error 0.7135181153076635
4
Root Mean Squared Error 0.801146703351678
Mean Absolute Error 0.6172466654275356

Average Root Mean Squared Error 0.843081693079764
Average Mean Absolute Error 0.6712080640845779
Final Results for Root Mean Squared Error 0.8736185924005034
Final Results for Mean Absolute Error 0.7287321153455143

 With K-Fold CV
For iadsed dominance SVR
KFold(n_splits=5, random_state=None, shuffle=False)
0
Root Mean Squared Error 0.9068787163319653
Mean Absolute Error 0.7282780700163187
1
Root Mean Squared Error 0.8929334902427101
Mean Absolute Error 0.7161642474405897
2
Root Mea

In [67]:
print("\n With K-Fold CV")
print("For iadsed dominance Deep Model")
fitted_model = apply_deep_model_with_kfold(df_iadsed_train_input, df_iadsed_train_actual_dominance, 48)
test_deep_model_with_kfold(fitted_model, df_iadsed_test_input, df_iadsed_test_actual_dominance)


 With K-Fold CV
For iadsed dominance Deep Model
0
Epoch 1/500
3/3 [==============================] - 1s 127ms/step - loss: 21.6848 - mean_squared_error: 21.6848 - mean_absolute_error: 4.5387 - val_loss: 18.2459 - val_mean_squared_error: 18.2459 - val_mean_absolute_error: 4.1582
Epoch 2/500
3/3 [==============================] - 0s 30ms/step - loss: 17.0395 - mean_squared_error: 17.0395 - mean_absolute_error: 3.9901 - val_loss: 14.5747 - val_mean_squared_error: 14.5747 - val_mean_absolute_error: 3.6808
Epoch 3/500
3/3 [==============================] - 0s 29ms/step - loss: 13.3603 - mean_squared_error: 13.3603 - mean_absolute_error: 3.4860 - val_loss: 11.4118 - val_mean_squared_error: 11.4118 - val_mean_absolute_error: 3.2035
Epoch 4/500
3/3 [==============================] - 0s 22ms/step - loss: 10.1903 - mean_squared_error: 10.1903 - mean_absolute_error: 2.9813 - val_loss: 8.7177 - val_mean_squared_error: 8.7177 - val_mean_absolute_error: 2.7247
Epoch 5/500
3/3 [=====================